In [114]:
# !pip install bs4
# !pip install selenium
# !pip install undetected_chromedriver
# !pip install fuzzywuzzy
# !pip install splinter

# Standard libraries for data handling and manipulation
import pandas as pd
import numpy as np
import os
import re

# Libraries for web scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import bs4 as bs
from bs4 import BeautifulSoup
from urllib.request import FancyURLopener  # Only if necessary
from fuzzywuzzy import fuzz, process  # For text matching, make sure you have python-Levenshtein for speed

# Miscellaneous utilities
import time
import datetime
from datetime import timedelta, date
import random
from random import choice



In [ ]:
#read companies list with earning conferencal call URLs.
df_all = pd.read_csv("C:\\Users\\accstu\\AppData\\Local\\Programs\\Python\\Python38\\Scripts\\transcript_list.csv")

user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
   'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel MaPc OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]


login_url = 'https://seekingalpha.com/account/login'
request_count = 0

def init_browser(user_agent):
    options = uc.ChromeOptions()
    options.add_argument(f"user-agent={user_agent}")
    driver = uc.Chrome(options=options)
    return driver

def login(driver, login_url, username, password):
    driver.get(login_url)
    time.sleep(50)  # wait time if there has CAPTCHA

    # Selenium API, use Xpath,ID or class.
    email_field = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[2]/form/div/div[1]/label/div/input')
    email_field.send_keys(username)
    password_field = driver.find_element(By.ID, "signInPasswordField")
    password_field.send_keys(password)

    # click login button
    login_button = driver.find_element(By.CSS_SELECTOR, 'button[data-test-id="sign-in-button"]')
    login_button.click()
    time.sleep(3)  # 等待登录过程完成

#anmsmax21111@mailto.plus
#Julia.123456
def getTextFromArticle(url):
    global request_count, current_browser
    if request_count >= 37: #number for requests, avoid CAPTCHA
        current_browser.quit()
        current_browser = init_browser(choice(user_agents))
        login(current_browser,login_url, "anmsmax21111@mailto.plus", "Julia.123456")  # 使用环境变量或其他方法获取这些值
        request_count = 0

    current_browser.get(url)
    time.sleep(3)
    html = current_browser.page_source
    soup = bs(html, "html.parser")
    texts = soup.find_all('p')
    total_doc = ''.join(text.text + '\n' for text in texts)
    
    request_count += 1
    return total_doc

current_browser = init_browser(choice(user_agents))
login(current_browser,login_url, "anmsmax21111@mailto.plus", "Julia.123456")  # 替换为正确的用户名和密码

#Save_articles_as_text
'''1.if index is in a list like, y=[1,2,3]'''
for index in y:
    row = df_all.iloc[index]
    
'''2.if index and row are in DataFrame'''    
# for index, row in df_ECT.iloc[2890:].iterrows():

    try:
        text = getTextFromArticle(row['url'])
        filename = row['title'] + '.txt'
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(text)
        time.sleep(1)
    except Exception as e:
        print(f"An error occurred at index {index}: {e}")
        continue

current_browser.quit()

In [56]:
y = [8680, 8721, 8732, 8734, 8735, 8742, 8746, 8750, 8878, 8886, 9369, 9393, 9398, 9479, 9510, 9577, 9578, 9584, 9600, 9604, 9646, 9668, 9758, 9983, 9992, 10070, 10083, 10085, 10086, 10087, 10090, 10091, 10093, 10094, 10095, 10096, 10099, 10101, 10102, 10103, 10129, 10198, 10211, 10336, 10364, 10366, 12400, 12404, 13411, 13615, 13616, 15831, 16988]

In [115]:
#df_all = pd.read_csv("C:\\Users\\accstu\\AppData\\Local\\Programs\\Python\\Python38\\Scripts\\transcript_list.csv")
'''
df_all: all URL, including PPT, Earning Call Transcripts
df_ECT: only include Earning Conference Call Transcripts URL
'''
df_ECT = df_all[df_all['title'].str.contains("Earnings Call Transcript")]

# creat a new column text
df_ECT["text"] = ""

In [117]:
'''
檢查缺失
1.Check if a file exists and whether its size is zero
'''
encodings = ['utf-8', 'latin1', 'ISO-8859-1', 'Windows-1252'] 
missing_files = []  

for index, row in df_ECT.iterrows():
    # earning call text 's save place
    filename = "D:\\Script\\" + row['title'] + '.txt'  #在我的D盤下Script文件夾
    file_exists = False
    
    #Check if a file exists and whether its size is zero
    if not os.path.exists(filename) or os.path.getsize(filename) == 0: #not size =0
        missing_files.append(index)
        df_ECT.at[index, 'text'] = pd.NA
        continue 

    for enc in encodings:
        try:
            with open(filename, 'r', encoding=enc) as file:
                text = file.read()
                df_ECT.at[index, 'text'] = text
                file_exists = True
                break  
                
        except FileNotFoundError:
            # 如果文件不存在，记录索引并设置为 NaN
            missing_files.append(index)
            df_ECT.at[index, 'text'] = pd.NA
            break  
            
        except UnicodeDecodeError:
            # 如果出现编码错误，就尝试下一种编码
            continue

    if not file_exists and index not in missing_files:
        missing_files.append(index)
print("Indexes of missing files:", missing_files)


df_ECT.to_csv("D:/Script/updated_transcript_list.tsv", sep='\t', index=False)


<ipython-input-117-a990f5d40636>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ECT["text"] = ""


Indexes of missing files: [8310]


In [119]:
'''
檢查缺失
2.Check text integrity
'''
def check_last_sentences(text, phrases):
    sentences = re.split(r'(?<=[.!?]) +', text)
    last_sentences = sentences[-10:] #last 10 sentences
    last_text = " ".join(last_sentences)
    
    for phrase in phrases:
        # 正则表达式来匹配词根和可能的常见后缀 ing,s,ed
        pattern = r'\b' + re.escape(phrase) + r'\b|\b' + re.escape(phrase) + r'(s|ing|ed)?\b'
        if re.search(pattern, last_text, re.IGNORECASE):
            return True
    return False

phrases = ["Thank you",'conclude','attend','conference','Thanks',
           "participation",'disconnect',
           'join','This article was written by',
           'following slide','Ladies and gentlemen','wonderful',
           'publish','concluded','attending','presentation']

#Don't have an account? Create free account
invalid_indices = []
for index, row in df_ECT.iterrows():
    text = row['text']
    if pd.notna(text):  # 确保 text 不是 NA 或 None
        if "The following slide" not in text and not check_last_sentences(text, phrases):
            invalid_indices.append(index)
print(invalid_indices)

[4019, 12400, 12404, 13411, 16988]


In [120]:
df_ECT

,title,url,date,ticker,text
2,3M Company (MMM) Q3 2023 Earnings Call Transcript,https://seekingalpha.com/article/4642981-3m-co...,2023-10-24,MMM,3M Company (NYSE:MMM) Q3 2023 Earnings Confere...
5,3M Company (MMM) Q2 2023 Earnings Call Transcript,https://seekingalpha.com/article/4619621-3m-co...,2023-07-25,MMM,3M Company (NYSE:MMM) Q2 2023 Earnings Confere...
7,3M Company (MMM) Q1 2023 Earnings Call Transcript,https://seekingalpha.com/article/4596435-3m-co...,2023-04-25,MMM,3M Company (NYSE:MMM) Q1 2023 Earnings Confere...
12,3M Company (MMM) Q4 2022 Earnings Call Transcript,https://seekingalpha.com/article/4571946-3m-co...,2023-01-24,MMM,3M Company (NYSE:MMM) Q4 2022 Earnings Confere...
15,3M Company (MMM) Q3 2022 Earnings Call Transcript,https://seekingalpha.com/article/4548991-3m-co...,2022-10-25,MMM,3M Company (NYSE:MMM) Q3 2022 Earnings Confere...
...,...,...,...,...,...
18588,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q2 ...",https://seekingalpha.com/article/4282234-zoeti...,2019-08-06,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q2 2019 Results Earnin..."
18591,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",https://seekingalpha.com/article/4259182-zoeti...,2019-05-02,ZTS,Zoetis Inc. (NYSE:ZTS) Q1 2019 Earnings Confer...
18592,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",https://seekingalpha.com/article/4241305-zoeti...,2019-02-14,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q4 2018 Earnings Confe..."
18594,Zoetis (ZTS) Q3 2018 Results - Earnings Call T...,https://seekingalpha.com/article/4217256-zoeti...,2018-11-01,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q3 2018 Earnings Call ..."


In [122]:
df_sandp500 = pd.read_csv('sandp500.csv')
df_sandp500_ECT = df_ECT.merge(df_sandp500, how='left', left_on='ticker', right_on='Symbol')
#Tab-Separated Values,TSV 文件使用制表符（\t）作为字段之间的分隔符
# df_sandp500_ECT.to_csv("D:/Script/sandp500_conference_earning_transcript.tsv", sep='\t', index=False)

In [123]:
df_sandp500_ECT

,title,url,date,ticker,text,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,3M Company (MMM) Q3 2023 Earnings Call Transcript,https://seekingalpha.com/article/4642981-3m-co...,2023-10-24,MMM,3M Company (NYSE:MMM) Q3 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,3M Company (MMM) Q2 2023 Earnings Call Transcript,https://seekingalpha.com/article/4619621-3m-co...,2023-07-25,MMM,3M Company (NYSE:MMM) Q2 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
2,3M Company (MMM) Q1 2023 Earnings Call Transcript,https://seekingalpha.com/article/4596435-3m-co...,2023-04-25,MMM,3M Company (NYSE:MMM) Q1 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
3,3M Company (MMM) Q4 2022 Earnings Call Transcript,https://seekingalpha.com/article/4571946-3m-co...,2023-01-24,MMM,3M Company (NYSE:MMM) Q4 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
4,3M Company (MMM) Q3 2022 Earnings Call Transcript,https://seekingalpha.com/article/4548991-3m-co...,2022-10-25,MMM,3M Company (NYSE:MMM) Q3 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10188,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q2 ...",https://seekingalpha.com/article/4282234-zoeti...,2019-08-06,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q2 2019 Results Earnin...",ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952
10189,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",https://seekingalpha.com/article/4259182-zoeti...,2019-05-02,ZTS,Zoetis Inc. (NYSE:ZTS) Q1 2019 Earnings Confer...,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952
10190,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",https://seekingalpha.com/article/4241305-zoeti...,2019-02-14,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q4 2018 Earnings Confe...",ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952
10191,Zoetis (ZTS) Q3 2018 Results - Earnings Call T...,https://seekingalpha.com/article/4217256-zoeti...,2018-11-01,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q3 2018 Earnings Call ...",ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


In [125]:
def split_presentation_QandA(text):
    # Check if the text is not null
    if pd.notna(text):
        presentation = []
        QandA = []
        in_presentation = False  # Initially not in the presentation section
        found_first_operator = False  # Flag to indicate the first 'Operator'

        for line in text.split('\n'):  # Splitting the text into lines
            if 'Operator' in line and not found_first_operator:
                # Found the first 'Operator', start of the presentation
                found_first_operator = True
                in_presentation = True
            elif 'Question-and-Answer Session' in line:
                # Found 'Question-and-Answer Session', start of the Q&A
                in_presentation = False
                continue  # Skip adding this line to any section
            
            if in_presentation:
                presentation.append(line)
            elif not in_presentation and found_first_operator:
                QandA.append(line)
        
        return ' '.join(presentation), ' '.join(QandA)
    else:
        # Return empty strings if the text is null
        return "", ""

# 现在应用这个函数到DataFrame的每行，并创建新列
df_sandp500_ECT[['Presentation', 'QandA']] = df_sandp500_ECT['text'].apply(lambda txt: pd.Series(split_presentation_QandA(txt)))


In [127]:
df_sandp500_ECT.to_csv("D:/Script/sandp500_conference_earning_transcript.tsv", sep='\t', index=False)

In [128]:
df_sandp500_ECT

,title,url,date,ticker,text,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Presentation,QandA
0,3M Company (MMM) Q3 2023 Earnings Call Transcript,https://seekingalpha.com/article/4642981-3m-co...,2023-10-24,MMM,3M Company (NYSE:MMM) Q3 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...
1,3M Company (MMM) Q2 2023 Earnings Call Transcript,https://seekingalpha.com/article/4619621-3m-co...,2023-07-25,MMM,3M Company (NYSE:MMM) Q2 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...
2,3M Company (MMM) Q1 2023 Earnings Call Transcript,https://seekingalpha.com/article/4596435-3m-co...,2023-04-25,MMM,3M Company (NYSE:MMM) Q1 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...
3,3M Company (MMM) Q4 2022 Earnings Call Transcript,https://seekingalpha.com/article/4571946-3m-co...,2023-01-24,MMM,3M Company (NYSE:MMM) Q4 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...
4,3M Company (MMM) Q3 2022 Earnings Call Transcript,https://seekingalpha.com/article/4548991-3m-co...,2022-10-25,MMM,3M Company (NYSE:MMM) Q3 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10188,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q2 ...",https://seekingalpha.com/article/4282234-zoeti...,2019-08-06,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q2 2019 Results Earnin...",ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952,Operator Welcome to the Second Quarter 2019 Fi...,Operator [Operator Instructions]. Our first qu...
10189,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",https://seekingalpha.com/article/4259182-zoeti...,2019-05-02,ZTS,Zoetis Inc. (NYSE:ZTS) Q1 2019 Earnings Confer...,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952,Operator Welcome to the First Quarter 2019 Fin...,Operator [Operator Instructions] We'll take ou...
10190,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",https://seekingalpha.com/article/4241305-zoeti...,2019-02-14,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q4 2018 Earnings Confe...",ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952,Operator Welcome to the Fourth Quarter and Ful...,Operator [Operator Instructions] We’ll take to...
10191,Zoetis (ZTS) Q3 2018 Results - Earnings Call T...,https://seekingalpha.com/article/4217256-zoeti...,2018-11-01,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q3 2018 Earnings Call ...",ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952,"Operator Good day, and welcome to the Third Qu...",Operator We'll take our first question from Jo...
